## Genius Setup

In [1]:
import sys
!{sys.executable} -m pip install lyricsgenius
!{sys.executable} -m pip install spotipy
!{sys.executable} -m pip install selenium
import pandas as pd
import requests
import lyricsgenius as genius
import spotipy
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
genius_client_id = 'Qeii-UKMUUwsAYCuuemsF7CZk7R2bFDhjP1FMkqpozASvYsA4wQDlLmd2JFM3Vhm'
genius_secret_id = 'fWhV7VtwAL-5D32Ga9Y0Kyh_VFEo7XK0Jf8-3RMcZ5EoMXqsDpmFWFFPyobnibfMXJuJ-GZe69YEuXR-36VGAA'
genius_client_access_token = '1QVL91jZbgdOIhvYhqKku_zkFDf6MHEyLpXEITHU2tE2HGvAqYIPInTrQaWcInD7'
app_website_url = 'http://cis545genius.com'
redirect_uri = 'cis545genius.com'
base_url = 'https://api.genius.com'
driver_path = '/Users/arnabsarker/Downloads/chromedriver.exe'
# driver_path = "/mnt/c/Users/jeffr/Downloads/chromedriver_win32/chromedriver.exe"
artist_url = 'https://www.ranker.com/crowdranked-list/the-greatest-rappers-of-all-time'
page_downs = 0
api = genius.Genius(genius_client_access_token)

## Graph Generation

In [2]:
import pandas as pd
import ast
import json

In [4]:
origin_df = pd.read_csv('data.csv')
data_df = origin_df[['genius_id', 'genius_data']]
origin_df = origin_df[['genius_id']].rename({'genius_id': 'song'})


In [5]:
def get_samples_of_song(song_id, song_data):
    song_json = ast.literal_eval(song_data)
    samples = song_json['_body']['song_relationships'][0]['songs']
    sample_ids = [];
    for song in samples:
        sample_ids.append(song['id'])
    return sample_ids
    

In [6]:
def insert_samples_into_graph(row):
    song_id = row[0]
    song_data = row[1]
    samples = get_samples_of_song(song_id, song_data)
    global graph_df
    global next_layer_df
    global num
    sample_dict_list = []
    for s in samples:
        num = num + 1
        sample_dict_list.append({'song': s})
    sample_df = pd.DataFrame(sample_dict_list)
    next_layer_df = next_layer_df.append(sample_df, ignore_index = True)
    sample_df = sample_df.rename(index=str, columns={"song": "sampled"})
    sample_df['sampler'] = song_id
    graph_df = graph_df.append(sample_df, ignore_index = True)
        

In [7]:
# samples for layer 1, from data.csv to avoid too many online calls
headers = {'Authorization': 'Bearer %s' % genius_client_access_token}
graph_df = pd.DataFrame(columns=['sampler', 'sampled'])

num = 0
curr_layer_df = data_df

next_layer_df = pd.DataFrame(columns=['song'])
curr_layer_df.apply(insert_samples_into_graph, axis = 1)
graph_df = graph_df.drop_duplicates()
next_layer_df = next_layer_df.drop_duplicates()
curr_layer_df = next_layer_df
    

In [8]:
def call_api_for_samples(song_id):
    try:
        base_url = 'https://api.genius.com'
        search_url = base_url + '/songs/' + str(song_id)
        params = {}
        response = requests.get(search_url, params=params, headers=headers)
        json = response.json()
        samples = json['response']['song']['song_relationships'][0]['songs']
        sample_ids = [];
        for song in samples:
            sample_ids.append(song['id'])
        return sample_ids
    except KeyError as e:
        return {}

In [9]:
def get_online_samples(song_id):
    samples = call_api_for_samples(song_id)
    global graph_df
    global next_layer_df
    sample_dict_list = []
    for s in samples:
        sample_dict_list.append({'song': s})
    sample_df = pd.DataFrame(sample_dict_list)
    next_layer_df = next_layer_df.append(sample_df, ignore_index = True)
    sample_df = sample_df.rename(index=str, columns={"song": "sampled"})
    sample_df['sampler'] = song_id
    graph_df = graph_df.append(sample_df, ignore_index = True)

In [10]:
depth = 4

for i in range(0, depth):
    next_layer_df = pd.DataFrame(columns=['song'])
    curr_layer_df.applymap(get_online_samples)
    graph_df = graph_df.drop_duplicates()
    next_layer_df = next_layer_df.drop_duplicates()
    curr_layer_df = next_layer_df

In [11]:
graph_df

,sampled,sampler
0,277313.0,3162643.0
1,1779130.0,1778100.0
2,696163.0,415468.0
3,67974.0,344010.0
4,230464.0,454761.0
5,3592672.0,88356.0
6,3592700.0,62993.0
7,771304.0,2399072.0
8,2195846.0,2354222.0
9,2308163.0,2399083.0


In [12]:
import sqlite3

engine = sqlite3.connect('Graph_DB')

graph_df.to_sql('graph_df', engine, if_exists='replace', index=True)

In [13]:
curr_layer_df

,song
0,3568802.0
1,2428317.0
2,3580531.0
3,499772.0
4,981201.0
5,2430409.0
6,118554.0
7,110516.0
8,118552.0
9,132290.0


In [4]:
play_df = pd.read_csv('data.csv')
play_df = play_df[['genius_id', 'genius_data']]

,genius_id,genius_data
2371,67272,"{'_body': {'annotation_count': 9, 'api_path': ..."


In [6]:
print(play_df[play_df['genius_id'] == 67272].genius_data.iloc[0])

{'_body': {'annotation_count': 9, 'api_path': '/songs/67272', 'description': {'dom': {'tag': 'root', 'children': [{'tag': 'p', 'children': ['“Beat It” is one of Jackson’s first forays into the rock genre, and talks about knowing when to back down from a fight. It is said to represent Jackson’s disdain for violence. The third single from ', {'tag': 'a', 'attributes': {'href': 'https://genius.com/artists/Michael-jackson'}, 'data': {'api_path': '/artists/835'}, 'children': ['Michael Jackson']}, '’s sixth solo album, ', {'tag': 'em', 'children': [{'tag': 'a', 'attributes': {'href': 'https://genius.com/albums/Michael-jackson/Thriller'}, 'data': {'api_path': '/albums/11769'}, 'children': ['Thriller']}]}, ' (1982), “Beat It” went Platinum and is an internationally recognized song that has greatly influenced pop culture.']}, '', {'tag': 'p', 'children': ['The song was ', {'tag': 'a', 'attributes': {'href': 'http://www.washingtontimes.com/news/2015/jun/10/berton-averre-interview-the-knacks-my-s